In [99]:
from collections import deque, defaultdict, Counter
from heapq import heapify, heappush, heappop
import numpy as np
from copy import deepcopy
import math
import time
from functools import cache, reduce, cmp_to_key
import graphviz
from itertools import product
import matplotlib.pyplot as plt
from bisect import bisect_left, bisect_right
import json
import os
import re
from typing import Any
from dataclasses import dataclass

In [100]:
def print_grid(grid: list[list[str]]):
    print("\n".join("".join(line) for line in grid))


def plot_grid(
    grid: list[list[str]],
    colors: dict[str, int],
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((len(grid), len(grid[0])))
    for r, row in enumerate(grid):
        for c, char in enumerate(row):
            if char in colors:
                arr[r, c] = colors[char]
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)


def plot_objects(
    object_lists: list[list[tuple[int, int]]],
    colors: list[int],
    x_limit: int,
    y_limit: int,
    save: bool = False,
    filepath: str = "images/plot.png",
) -> None:
    arr = np.zeros((y_limit, x_limit))
    for objects, color in zip(object_lists, colors):
        for obj in objects:
            arr[y_limit - 1 - obj[1], obj[0]] = color
    plt.xticks([])
    plt.yticks([])
    if save:
        plt.imsave(filepath, arr)
    else:
        plt.imshow(arr)

In [101]:
dirs4 = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dirs8 = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

today = os.path.basename(globals()["__vsc_ipynb_file__"]).split(".")[0]  # + "_ex"
today

'day21'

In [102]:
def get_lines() -> list[str]:
    lines = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            lines.append(line.rstrip())
    return lines


def get_grid() -> list[list[str]]:
    grid = []
    with open(f"./data/{today}.txt") as f:
        while line := f.readline():
            grid.append([c for c in line.rstrip()])
    return grid


def parse_nums(s: str) -> list[int]:
    return [int(x) for x in re.findall(r"-?\d+", s)]


def get_nums() -> list[list[int]]:
    lines = get_lines()
    return [parse_nums(line) for line in lines]


def is_inside_grid(coords: tuple[int, int], grid: list[list[Any]]) -> bool:
    return coords[0] in range(len(grid)) and coords[1] in range(len(grid[0]))

In [103]:
codes = get_lines()

In [104]:
def shortest_base(start: str, end: str) -> set[str]:
    coords = {
        "A": (3, 1),
        "0": (2, 1),
        "1": (1, 2),
        "2": (2, 2),
        "3": (3, 2),
        "4": (1, 3),
        "5": (2, 3),
        "6": (3, 3),
        "7": (1, 4),
        "8": (2, 4),
        "9": (3, 4),
    }
    start_coords = coords[start]
    end_coords = coords[end]

    res = set()
    if start_coords[0] != 1 or end_coords[1] != 1:
        res.add(
            ("^" if end_coords[1] > start_coords[1] else "v")
                * abs(end_coords[1] - start_coords[1])
            + (">" if end_coords[0] > start_coords[0] else "<")
            * abs(end_coords[0] - start_coords[0]) + "A"
        )
    if start_coords[1] != 1 or end_coords[0] != 1:
        res.add(
            (">" if end_coords[0] > start_coords[0] else "<")
            * abs(end_coords[0] - start_coords[0])
            + ("^" if end_coords[1] > start_coords[1] else "v")
            * abs(end_coords[1] - start_coords[1]) + "A"
        )
    return res


def shortest_moves_first_robot(seq: str) -> list[str]:
    res = []
    pos = "A"
    for c in seq:
        res.append(shortest_base(pos, c))
        pos = c

    return list(reduce(lambda x, y: x+y, moves) for moves in product(*res))

In [105]:
shortest_moves_first_robot("029A")

['<A^A^^>AvvvA', '<A^A>^^AvvvA']

In [106]:
def shortest_later(start: str, end: str) -> set[str]:
    coords = {
        "<": (1, 1),
        "v": (2, 1),
        ">": (3, 1),
        "^": (2, 2),
        "A": (3, 2),
    }
    start_coords = coords[start]
    end_coords = coords[end]

    res = set()
    res.add(("^" if end_coords[1] > start_coords[1] else "v") * abs(
        end_coords[1] - start_coords[1]
    ) + (">" if end_coords[0] > start_coords[0] else "<") * abs(
        end_coords[0] - start_coords[0]
    ) + "A")
    if end != "<":
        res.add((">" if end_coords[0] > start_coords[0] else "<") * abs(
            end_coords[0] - start_coords[0]
        ) + ("^" if end_coords[1] > start_coords[1] else "v") * abs(
            end_coords[1] - start_coords[1]
        ) + "A")
    return res

def expand_seq(seq: str) -> list[str]:
    res = []
    pos = "A"
    for c in seq:
        res.append(shortest_later(pos, c))
        pos = c
    
    return list(reduce(lambda x, y: x+y, moves) for moves in product(*res))

def shortest_moves_later_robot(seqs: list[str]) -> list[str]:
    res = []
    for seq in seqs:
        tmp = expand_seq(seq)
        res.extend(tmp)
    return res

In [108]:
def possible_paths(seq: str, depth: int) -> list[str]:
    if depth == 0:
        return shortest_moves_first_robot(seq)
    seqs = possible_paths(seq, depth-1)
    return shortest_moves_later_robot(seqs)

In [109]:
s = 0
for seq in codes:
    l = len(sorted(possible_paths(seq, 2), key=len)[0])
    s += int(seq[:-1]) * l
s  # 162740

162740